In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import glob
from PIL import Image
from sklearn.model_selection import train_test_split
import torchvision
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from tensorboardX import SummaryWriter
from torch.optim.lr_scheduler import LambdaLR, StepLR

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class Config:

    batch_size = 64
    epochs = 300
    lr = 1e-3
    
    channel = 3

    height = 32
    width = 32

    data_root = '../dataset/cifar10'

    dropout_rate = 0.1
    attn_dropout = 0

    patch_size = 4
    num_patches = int((height * width) / (patch_size ** 2))

    layers = 12
#     embedding_d = 768
    embedding_d = 512
    mlp_size = 1024
    heads = 8
    
    dimhead = 512

    num_classes = 10
    
    log_f = 100

In [4]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
])

In [5]:
trainset = torchvision.datasets.CIFAR10(root=Config.data_root, train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=Config.batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=Config.data_root, train=False,
                                       download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=Config.batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.num_patches = (Config.height * Config.width) // Config.patch_size ** 2
        self.patch_d = Config.channel * Config.patch_size * Config.patch_size
        
        self.patch_embedding = PatchEmbedding()
        self.class_token = nn.Parameter(torch.randn(1, 1, Config.embedding_d), requires_grad=True)
        self.position_embedding = nn.Parameter(torch.randn(1, Config.num_patches + 1, Config.embedding_d), requires_grad=True)
        self.cls_token = nn.Parameter(torch.randn(1, 1, Config.dimhead))
        self.dropout = nn.Dropout(Config.dropout_rate)
        
        self.transformer = Transformer()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(Config.dimhead),
            nn.Linear(Config.dimhead, Config.num_classes))
        
        
    def forward(self, x):
        x = self.patch_embedding(x)
        class_token = self.class_token.expand(Config.batch_size, -1, -1)
        x = torch.cat((class_token, x), dim=1)
        x = self.position_embedding + x
        x = self.dropout(x)
        
        x = self.transformer(x)
        
        x = x[:, 0]

        x = self.mlp_head(x)
        
        return x
    
    
class Transformer(nn.Module):
    def __init__(self, dim=Config.dimhead, mlp_dim=512):
        super().__init__()
        
        self.layers = nn.ModuleList([])
        for _ in range(Config.layers):
            self.layers.append(nn.ModuleList([MABlock(), FeedForward()]))
                               
    def forward(self, x):
        
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
            
        return x
    
class MABlock(nn.Module):

    def __init__(self):
        super(MABlock, self).__init__()

        self.layer_norm = nn.LayerNorm(Config.embedding_d)
        self.multihead_attention = nn.MultiheadAttention(Config.embedding_d, Config.heads, Config.attn_dropout, batch_first=True)

    def forward(self, x):
        x = self.layer_norm(x)
        x, _ = self.multihead_attention(x, x, x, need_weights=False)

        return x
        
        
class PatchEmbedding(nn.Module):

    def __init__(self):
        super(PatchEmbedding, self).__init__()

        self.conv = nn.Conv2d(in_channels=Config.channel,
                              out_channels=Config.embedding_d,
                              kernel_size=Config.patch_size,
                              stride=Config.patch_size)

        self.flatten = nn.Flatten(start_dim=2, end_dim=3)

    def forward(self, x):
        x = self.conv(x)  # ([b, 768, 4, 4])
        x = self.flatten(x)  # ([b, 768, 16])

        return x.permute(0, 2, 1)
        
        
class FeedForward(nn.Module):
    
    def __init__(self):
        super(FeedForward, self).__init__()

        self.layer_norm = nn.LayerNorm(Config.embedding_d)
        self.mlp = nn.Sequential(
            nn.Linear(Config.embedding_d, Config.mlp_size),
            nn.GELU(),
            nn.Dropout(Config.dropout_rate),
            nn.Linear(Config.mlp_size, Config.embedding_d),
            nn.Dropout(Config.dropout_rate)
        )

    def forward(self, x):
        x = self.layer_norm(x)
        x = self.mlp(x)

        return x

In [8]:
model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=Config.lr)
criterion = nn.CrossEntropyLoss()

In [9]:
scheduler = StepLR(optimizer, 5)

In [10]:
warmup_scheduler = LambdaLR(optimizer, lr_lambda=lambda step: min(1.0, step / 10000))
step_scheduler = StepLR(optimizer, step_size=5, gamma=.9)

In [11]:
writer = SummaryWriter()

In [12]:
def test(epoch):

    model.eval()
    
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        
        for batch_idx, (inputs, targets) in enumerate(tqdm(testloader)):
            
            if Config.batch_size != len(inputs):
                inputs = inputs.repeat(Config.batch_size // inputs.size(0) + 1, 1, 1, 1)[:Config.batch_size]
                targets = targets.repeat(Config.batch_size // targets.size(0) + 1)[:Config.batch_size]
            
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total

    print(f'Epoch {epoch} val loss: {test_loss:.5f}, test acc: {(acc):.5f}')

    return test_loss, acc

In [13]:
def train(epoch):
    
    model.train()
    
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader)):
        
        if Config.batch_size != len(inputs):
            inputs = inputs.repeat(Config.batch_size // inputs.size(0) + 1, 1, 1, 1)[:Config.batch_size]
            targets = targets.repeat(Config.batch_size // targets.size(0) + 1)[:Config.batch_size]
        
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        acc = 100.*correct/total
        
        if batch_idx % Config.log_f == 0:
            print(f'Epoch {epoch}, batch index {batch_idx} || train loss: {train_loss/(batch_idx+1)}, train acc: {acc}')

    return train_loss/(batch_idx+1), acc

In [ ]:
for epoch in range(Config.epochs):

    train_loss, train_acc = train(epoch)
    test_loss, test_acc = test(epoch)
    
    scheduler.step()

    writer.add_scalar('train/loss', train_loss, epoch)
    writer.add_scalar('train/acc', train_acc, epoch)
    writer.add_scalar('test/loss', test_loss, epoch)
    writer.add_scalar('test/acc', test_acc, epoch)

  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 0, batch index 0 || train loss: 2.5789201259613037, train acc: 7.8125
Epoch 0, batch index 100 || train loss: 2.471000834266738, train acc: 10.055693069306932
Epoch 0, batch index 200 || train loss: 2.4677635686314523, train acc: 10.051305970149254
Epoch 0, batch index 300 || train loss: 2.464347756186197, train acc: 10.101744186046512
Epoch 0, batch index 400 || train loss: 2.464761009240091, train acc: 9.955579800498754
Epoch 0, batch index 500 || train loss: 2.4635067419139687, train acc: 10.04553393213573
Epoch 0, batch index 600 || train loss: 2.462752358092246, train acc: 10.004159733777039
Epoch 0, batch index 700 || train loss: 2.4627684104799714, train acc: 10.065977175463624


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 0 val loss: 385.68193, test acc: 9.95223


  0%|          | 0/782 [00:00<?, ?it/s]

...

Epoch 86, batch index 0 || train loss: 2.386270523071289, train acc: 12.5
Epoch 86, batch index 100 || train loss: 2.466677960782948, train acc: 10.133044554455445
Epoch 86, batch index 200 || train loss: 2.4635151239176887, train acc: 9.934701492537313
Epoch 86, batch index 300 || train loss: 2.464698489718263, train acc: 10.018687707641195
Epoch 86, batch index 400 || train loss: 2.4652516574336407, train acc: 10.07637157107232
Epoch 86, batch index 500 || train loss: 2.4633531884518924, train acc: 10.082959081836327
Epoch 86, batch index 600 || train loss: 2.463544060108864, train acc: 10.053556572379367
Epoch 86, batch index 700 || train loss: 2.4632388525104387, train acc: 10.097182596291013


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 86 val loss: 385.68193, test acc: 9.95223


  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 87, batch index 0 || train loss: 2.4388186931610107, train acc: 18.75
Epoch 87, batch index 100 || train loss: 2.4618716098294398, train acc: 10.504331683168317
